In [2]:
import sys

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

print(sys.executable)


/opt/anaconda3/envs/mistral-chatbot/bin/python


In [3]:
model_name = "Qwen/Qwen3-1.7B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",

)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [4]:
print(model.device)


mps:0


In [9]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is a Large Language Model?"}
]

chat = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False
)

inputs = tokenizer(chat, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7,
    top_p=0.8,
    top_k=20
)

response_text = tokenizer.decode(output[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)
print("Answer:", response_text.strip())



Answer: A **Large Language Model (LLM)** is a type of artificial intelligence (AI) model that is trained on vast amounts of text data to understand and generate human-like text. These models are designed to perform a wide range of tasks, such as
